In [ ]:
#Description
print("""This code uses Biopython to search DNA sequences from FASTA files(.fa) across the BLAST database, returning species information, genome alignments, and sequence similarities""")
print("\n")
print("Note: This program can return a maximum of 50 results per record.")

#Imports
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import time

#Takes path of FASTA file
file_path=input("Please input the path of your FASTA file(.fa) here: ")

#Takes the contents of the file into a string
fasta_string=open(file_path).read()

#Splits the file into records
records=fasta_string.split(">")

#Removes the blank string
records.remove(records[0])

#Re-adds the ">" sign in each record
for record in records:
    records[records.index(record)] = ">"+record

#Asks which record should be searched
queries = input(f"""Your file has {len(records)} records. If you want to search multiple records, put commas between each record number. Please input the number of the records you want to search: """).split(",")

#Turns the strings to integers
for query in queries:
    queries[queries.index(query)]=int(query)

#Requests what algorithm to use
alg=input("Which algorithm do you want to use, blastn, blastp, blastx, tblastn, or tblastx? ").lower()
if alg=="blastn":
    mb=input("Do you want to use megablast as well? Y or N ").upper()
    if mb=="Y":
        mb=True
    elif mb=="N":
        mb=None
else:
    mb=None

#Searches using qblast
for query in queries:
    start_time=time.perf_counter()
    print("\n")
    print("\n")
    print("\n")
    print(f"**** Searching Record Number {query} ****")
    result_handle=NCBIWWW.qblast("blastn","nt",records[query-1], megablast=mb)
    
    #Reads these results into the record
    blast_record=NCBIXML.read(result_handle)

    #Sets the e value
    e_value_thresh=0.05
    
    #Checks if there are alignments to print
    #Shows alignments
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < e_value_thresh:
                print("****Alignment****")
                print(f"Sequence: {alignment.title}")
                print(f"Length: {alignment.length}")
                print(f"E Value: {hsp.expect}")
                if hsp.query==hsp.sbjct:
                    print("Exact Match!")
                    end_time=time.perf_counter()
                    execution_time=abs(end_time-start_time)
                    print(f"Execution time: {round(execution_time)} seconds")
                    print("\n")
                else:
                    print(hsp.query)
                    print(hsp.match)
                    print(hsp.sbjct)
                    end_time=time.perf_counter()
                    execution_time=abs(end_time-start_time)
                    print(f"Execution time: {round(execution_time)} seconds")
                    print("\n")
                    
#Finishes results
print("No other significant similarities found")